<a href="https://colab.research.google.com/github/41371115h-dot/114-class-thursday2-4/blob/main/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80%E4%BD%9C%E6%A5%AD%E5%85%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ===============================
# 📚 課表 AI 讀書建議系統
# ===============================

import os
import pandas as pd
import gradio as gr
import google.generativeai as genai
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.colab import auth
from google.auth import default

# ====== 🔐 Google Sheets 驗證 ======
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# ====== 📘 讀取 Google Sheet ======
sheet_url = "https://docs.google.com/spreadsheets/d/19xho0MmcY76NTco9T1bcyzcM7byBKpWtObOgZK12674/edit?usp=sharing"
spreadsheet = gc.open_by_url(sheet_url)
worksheet = spreadsheet.sheet1

df = get_as_dataframe(worksheet, evaluate_formulas=True)
df = df.dropna(how="all")  # 移除空行
df = df.fillna("")          # 空格補空字串

# ====== 🤖 連接 Gemini API ======
from google.colab import userdata

# 從 Colab Secrets 中獲取 API 金鑰
api_key = userdata.get('ella4')

# 使用獲取的金鑰配置 genai
genai.configure(api_key=api_key)

# ====== 🎯 AI 建議生成函數 ======
def generate_study_advice(goal):
    prompt = f"""
    這是我的這週課表：
    {df.to_string(index=False)}

    我的這週學習目標是：{goal}

    請根據課表和目標，幫我：
    1. 提供具體的學習建議（每天該怎麼安排）
    2. 推薦 3 支適合的 YouTube 影片（附上關鍵字，不用真實連結）
    """

    model = genai.GenerativeModel("gemini-1.5-pro")
    response = model.generate_content(prompt)
    return response.text.strip()

# ====== ✍️ 寫回 Google Sheet ======
def write_back_to_sheet(goal, advice):
    try:
        sheet = spreadsheet.worksheet("AI建議")
    except:
        sheet = spreadsheet.add_worksheet(title="AI建議", rows=100, cols=3)
        sheet.append_row(["時間", "學習目標", "AI建議"])

    import datetime
    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
    sheet.append_row([now, goal, advice])

# ====== 🧠 Gradio 介面函數 ======
def main(goal):
    advice = generate_study_advice(goal)
    write_back_to_sheet(goal, advice)
    return df, advice

# ====== 🎨 Gradio 介面設計 ======
with gr.Blocks() as demo:
    gr.Markdown("## 🧭 我的課表與 AI 讀書建議系統")

    gr.Markdown("### 📅 你的課表")
    table = gr.Dataframe(value=df, label="課表（自動從 Google Sheet 讀取）")

    goal_input = gr.Textbox(label="🎯 請輸入這週的學習目標", placeholder="例如：加強程式語言理解、複習線性代數")
    output_advice = gr.Textbox(label="💡 AI 給的學習建議", lines=10)
    submit_btn = gr.Button("✨ 生成學習建議")

    submit_btn.click(fn=main, inputs=goal_input, outputs=[table, output_advice])

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dc205c072e6859ef30.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
